<a href="https://colab.research.google.com/github/Untick/IrisID_gr1/blob/Daria-Petrova-folder/Iris_ID_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ТЗ**

**Задача:** идентификация пациента по радужной оболочке глаза (Iris ID).

**Тип задачи:** классификация.
Количество классов: равно удвоенному количеству пациентов (левый и
правый глаза).

**Датасет:** цветные изображения переднего сегмента глаз в формате png с
разрешением 1920x1080. В качестве источника датасета задействованы 10
человек. По 50 снимков на каждый глаз (всего 1000 снимков). При
необходимости количество снимков можно будет увеличить.
Задействованных людей можно будет использовать для тестирования
обученной сети.

**Особенности:** возможность дообучения на датасете, содержащем новые
классы.

**Точность:** FRR (ложный недопуск) ошибка не более 3%, FAR (ложный
допуск) ошибка не более 3%, идентификация: точность не менее 97%.

**Описание идентификации:**

На вход системы идентификации подается изображение переднего сегмента
правого глаза пациента. С помощью сети сегментации (разработанной на
предыдущем этапе) извлекается область радужной оболочки и зрачка.
Вычисляется соотношение диаметра зрачка к диаметру радужной оболочки.
Если это соотношение более, чем 0,5 (слишком расширенный зрачок), то
система идентификации возвращает «невозможность идентификации», т. е.
область радужной оболочки слишком мала для возможности точной
идентификации. Эта область подается на сеть классификации. Сеть
классификации возвращает вероятность класса для правого глаза. Затем
подается изображение переднего сегмента левого глаза пациента и снова
возвращается вероятность класса. Если вероятность класса хотя бы для
одного из двух глаз ниже 97%, то система идентификации возвращает
«недопуск», т. е. отсутствие пациента в системе идентификации.

# **Описание НС**

**Имеется:** папка IrisID на Google Drive, в которую будут загружаться датасеты для обучения нейросети в виде архивов с наименованиями пациентов.

Данная программа обновляет и обрабатывает новые файлы и папки в папке IrisID на Google Drive. Затем происходит аугментация изображений, разделение датасета на обучающую и тестовую выборки, и, при необходимости, повторная проверка наличия новых файлов и папок для последующей обработки. Этот процесс позволяет автоматически интегрировать новые данные в существующий датасет и готовить его для обучения и дообучения нейросети.

## **I. Работа с новыми датасетами.**

Этот этап необходим для проверки наличия новых датасетов в исходной папке IrisID на Google Drive и добавления их в директорию Google Colab.

**Подключение к Google Drive.**

Программа начинает с подключения к Google Drive с помощью drive.mount('/content/drive'). После успешного подключения получаем доступ к файлам и папкам на Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**Импортирование папки с датасетами из Google Drive в директорию Google Colab.**

Затем программа импортирует папку IrisID из Google Drive в директорию Google Colab. Для этого используется shutil.copytree(drive_folder_path, colab_folder_path), где drive_folder_path - путь к папке IrisID на Google Drive, а colab_folder_path - путь к папке IrisID в директории Google Colab. С помощью os.path.exists(colab_folder_path) проверяется условие, существует ли уже папка IrisID в директории Google Colab. Если папка уже существует, выводится сообщение "Папка IrisID уже существует в директории Google Colab". Если папки нет, то выполняется копирование файлов из Google Drive в Google Colab.






In [ ]:
import shutil
import os

# Путь к папке на Google Drive, в которую будут загружаться новые данные
drive_folder_path = '/content/drive/MyDrive/IrisID'

# Путь к папке в директории Google Colab
colab_folder_path = '/content/IrisID'

if not os.path.exists(colab_folder_path):
    # Копирование файлов из Google Drive в Google Colab
    os.makedirs(colab_folder_path)
    
    # Копирование файлов из Google Drive в Google Colab
for file_name in os.listdir(drive_folder_path):
    src = os.path.join(drive_folder_path, file_name)
    dst = os.path.join(colab_folder_path, file_name)
    shutil.copy(src, dst)

else:
    print("Папка IrisID уже существует в директории Google Colab.")

**Проверка наличия новых архивов в папке IrisID в директории Google Colab.**

In [ ]:
archives = [f for f in os.listdir(drive_folder_path) if f.endswith('.zip')]

if len(archives) == 0:
    print("Архивы отсутствуют в папке IrisID на Google Drive.")
else:
    print("Найдены следующие архивы в папке IrisID на Google Drive:")
    print('\n'.join(archives))

# Путь к файлу, в котором будет храниться список предыдущих архивов
# (с момента последнего пополенения папки IrisID на Google Drive)
previous_archives_file = '/content/previous_archives.txt'

# Проверка наличия предыдущего списка архивов
if os.path.exists(previous_archives_file):
    # Загрузка предыдущего списка архивов
    with open(previous_archives_file, 'r') as file:
        previous_archives = file.read().splitlines()
else:
    previous_archives = []

# Проверка наличия новых архивов и отображение их в виде списка
new_archives = [archive for archive in archives if archive not in previous_archives]

print("Среди них новых архивов:", end=" ")
if len(new_archives) == 0:
    print("нет.")
else:
    print(f"{len(new_archives)}")
    print('\n'.join(new_archives))

# Объединение текущего списка архивов с предыдущими
updated_archives = previous_archives + archives

# Сохранение обновлённого списка архивов
with open(previous_archives_file, 'w') as file:
    file.write('\n'.join(updated_archives))

**Распаковка новых архивов и сортировка изображений по папкам классов.**

Новые архивы из папки IrisID на Google Drive извлекаются с помощью функции extract_archives и размещаются в папку IDs в директории Google Colab. Код проверяет наличие папки с именем архива внутри папки IDs перед извлечением. Если папка уже существует, архив пропускается. 

In [ ]:
import zipfile
import glob

def extract_archives():
    # Пути к папкам
    ids_path = os.path.join(colab_folder_path, 'IDs')

    # Распаковка архивов
    archives = glob.glob(os.path.join(colab_folder_path, '*.zip'))
    for archive in archives:
        archive_name = os.path.splitext(os.path.basename(archive))[0]
        extraction_path = os.path.join(ids_path, archive_name)

        # Проверка наличия папки с именем архива внутри папки IDs
        if not os.path.exists(extraction_path):
            with zipfile.ZipFile(archive, 'r') as zip_ref:
                # Получение списка файлов в архиве
                file_list = zip_ref.namelist()

                # Проверка наличия основной директории в архиве
                has_main_directory = False
                for file in file_list:
                    if '/' in file:
                        has_main_directory = True
                        break

                # Если основная директория отсутствует, создаём папку с названием архива внутри IDs
                if not has_main_directory:
                    extraction_path = os.path.join(ids_path, archive_name)
                    os.makedirs(extraction_path)

                # Извлечение файлов
                zip_ref.extractall(extraction_path)
            os.remove(archive)
        else:
            print(f"Архив {archive_name} уже распакован. Удаляю архив...")
            os.remove(archive)

**Полная очистка папки IDs от содержимого, при необходимости.**

Этот код использует функцию shutil.rmtree() для удаления содержимого папки IDs. Затем функция os.makedirs() создаёт пустую папку IDs, чтобы была возможность начать с чистого листа при необходимости.

In [ ]:
import shutil

def clear_ids_folder():
      
    # Удаление содержимого папки IDs
    shutil.rmtree(colab_folder_path)
    
    # Пересоздание пустой папки IDs
    os.makedirs(colab_folder_path)

    extract_archives()

# Вызов функции для очистки папки IDs
clear_ids_folder()

## **II. Обработка изображений.**

### **Аугментация изображений.**

### **Перемешивание и разделение обновлённого датасета на train и test выборки.**

# **ДОПОЛНИТЕЛЬНО: автоматическое повторение этапов I и II.**

Проверка наличия новых датасетов в папке IrisID на Google Drive, и, при обнаружении, - добавление их в директорию Google Colab, аугментация и перераспределение в train и test выборки.

Подключение к Google Drive.

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Функция process_new_archives() содержит все шаги для обработки новых архивов и выполнения всех последующих действий.

In [ ]:
#@title
import shutil
import os
import zipfile
import glob

def process_new_archives():

    # Шаг 1: Поиск новых архивов в папке IrisID на Google Drive

    # Путь к папке на Google Drive, в которую будут загружаться новые данные
    drive_folder_path = '/content/drive/MyDrive/IrisID'

    # Путь к папке в директории Google Colab
    colab_folder_path = '/content/IrisID'

    if not os.path.exists(colab_folder_path):
        # Копирование файлов из Google Drive в Google Colab
        os.makedirs(colab_folder_path)
        
        # Копирование файлов из Google Drive в Google Colab
    for file_name in os.listdir(drive_folder_path):
        src = os.path.join(drive_folder_path, file_name)
        dst = os.path.join(colab_folder_path, file_name)
        shutil.copy(src, dst)

    else:
        print("Папка IrisID уже существует в директории Google Colab.")

    # Шаг 2: Распаковка архивов в папку IDs

    archives = [f for f in os.listdir(drive_folder_path) if f.endswith('.zip')]

    if len(archives) == 0:
        print("Архивы отсутствуют в папке IrisID на Google Drive.")
    else:
        print("Найдены следующие архивы в папке IrisID на Google Drive:")
        print('\n'.join(archives))

    # Путь к файлу, в котором будет храниться список предыдущих архивов
    # (с момента последнего пополенения папки IrisID на Google Drive)
    previous_archives_file = '/content/previous_archives.txt'

    # Проверка наличия предыдущего списка архивов
    if os.path.exists(previous_archives_file):
        # Загрузка предыдущего списка архивов
        with open(previous_archives_file, 'r') as file:
            previous_archives = file.read().splitlines()
    else:
        previous_archives = []

    # Проверка наличия новых архивов и отображение их в виде списка
    new_archives = [archive for archive in archives if archive not in previous_archives]

    print("Среди них новых архивов:", end=" ")
    if len(new_archives) == 0:
        print("нет.")
    else:
        print(f"{len(new_archives)}")
        print('\n'.join(new_archives))

    # Объединение текущего списка архивов с предыдущими
    updated_archives = previous_archives + archives

    # Сохранение обновлённого списка архивов
    with open(previous_archives_file, 'w') as file:
        file.write('\n'.join(updated_archives))

    def extract_archives():
        # Пути к папкам
        ids_path = os.path.join(colab_folder_path, 'IDs')

        # Распаковка архивов
        archives = glob.glob(os.path.join(colab_folder_path, '*.zip'))
        for archive in archives:
            archive_name = os.path.splitext(os.path.basename(archive))[0]
            extraction_path = os.path.join(ids_path, archive_name)

            # Проверка наличия папки с именем архива внутри папки IDs
            if not os.path.exists(extraction_path):
                with zipfile.ZipFile(archive, 'r') as zip_ref:
                    # Получение списка файлов в архиве
                    file_list = zip_ref.namelist()

                    # Проверка наличия основной директории в архиве
                    has_main_directory = False
                    for file in file_list:
                        if '/' in file:
                            has_main_directory = True
                            break

                    # Если основная директория отсутствует,
                    # создаём папку с названием архива внутри IDs
                    if not has_main_directory:
                        extraction_path = os.path.join(ids_path, archive_name)
                        os.makedirs(extraction_path)

                    # Извлечение файлов
                    zip_ref.extractall(extraction_path)
                os.remove(archive)
            else:
                print(f"Архив {archive_name} уже распакован. Удаляю архив...")
                os.remove(archive)

# Функция для полной очистки папки IDs от содержимого
def clear_ids_folder():
      
    # Удаление содержимого папки IDs
    shutil.rmtree(colab_folder_path)
    
    # Пересоздание пустой папки IDs
    os.makedirs(colab_folder_path)

    extract_archives()

    # Шаг 3: Аугментация новых датасетов

    # Шаг 4: Разделение аугментированных датасетов на выборки


В ячейке ниже, где вызывается функция process_new_archives(), она будет выполнять все шаги автоматически.

In [ ]:
# Вызов функции для обработки новых архивов и выполнения всех шагов
process_new_archives()

Папка IrisID уже существует в директории Google Colab.
Найдены следующие архивы в папке IrisID на Google Drive:
iris1_8.zip
iris25_32.zip
iris9_16.zip
iris17_24.zip
Среди них новых архивов: 4
iris1_8.zip
iris25_32.zip
iris9_16.zip
iris17_24.zip


Всё содержимое папки IDs можно очистить, если была допущена какая-либо ошибка...

In [ ]:
# Вызов функции для очистки папки IDs
clear_ids_folder()

... И проделать этапы I-II заново.